In [2]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import PIL
import cv2

import pathlib

Download Dataset in our Local Disk

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it

Read Data from Local Disk

In [3]:
data_dir = pathlib.Path('./datasets/flower_photos/')

flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

Store All Flowers in Python List

In [4]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

Convert into Numpy Array

In [5]:
X = np.array(X)
y = np.array(y)

Train, Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=55)

Scale Image

In [7]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

Imporve Model Accuracy with Data Augmentation

In [8]:
img_height = 180
img_width = 180

data_augmentation = keras.Sequential(
  [ 
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

Train Model

In [9]:
num_classes = 5

model = Sequential([
  data_augmentation,
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=5) 

Epoch 1/5
86/86 [==============================] - 78s 859ms/step - loss: 2.8368 - accuracy: 0.3710
Epoch 2/5
86/86 [==============================] - 77s 900ms/step - loss: 1.1421 - accuracy: 0.5280
Epoch 3/5
86/86 [==============================] - 71s 825ms/step - loss: 1.0460 - accuracy: 0.5923
Epoch 4/5
86/86 [==============================] - 64s 750ms/step - loss: 0.9752 - accuracy: 0.6188
Epoch 5/5
86/86 [==============================] - 66s 769ms/step - loss: 0.9412 - accuracy: 0.6323


In [10]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 4s 142ms/step - loss: 1.2182 - accuracy: 0.5795


[1.2182027101516724, 0.5795207023620605]